## Install required packages

In [ ]:
!pip install qiskit qiskit-aer numpy

In [ ]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit import qasm2
import matplotlib.pyplot as plt


def normalize_pixel_value(pixel, min_val, max_val):
    """Normalize pixel value to [0, 1] range"""
    if max_val == min_val:
        return 0.5
    return (float(pixel) - float(min_val)) / (float(max_val) - float(min_val))

def isqr_encoding(normalized_pixel):
    """ISQR encoding"""
    return 2.0 * np.arccos(np.sqrt(normalized_pixel))  # Exact from your code

def create_quantum_circuit(pixel_value, min_val, max_val):
    """
    Create quantum circuit that EXACTLY replicates your Python code

    Quantum equivalent: Ry(2θ)|0⟩ = cos(θ)|0⟩ + sin(θ)|1⟩
    """

    # Calculate exactly as in your code
    normalized = normalize_pixel_value(pixel_value, min_val, max_val)
    theta = isqr_encoding(normalized)

    print(f"Calculations from YOUR code:")
    print(f"  Pixel value: {pixel_value}")
    print(f"  Normalized: {normalized:.4f}")
    print(f"  Theta (radians): {theta:.4f}")
    print(f"  cos(theta): {np.cos(theta):.4f}")
    print(f"  sin(theta): {np.sin(theta):.4f}")
    print(f"  Probability |0⟩: {np.cos(theta)**2:.4f}")
    print(f"  Probability |1⟩: {np.sin(theta)**2:.4f}")

    # Create quantum circuit that replicates this
    # 1 qubit, 1 classical bit
    qc = QuantumCircuit(1, 1, name=f"pixel_{pixel_value}")

    # Apply Ry(2θ) to get state: cos(θ)|0⟩ + sin(θ)|1⟩
    # Note: Ry(2θ)|0⟩ = cos(θ)|0⟩ + sin(θ)|1⟩ exactly matches your i_state
    qc.ry(2 * theta, 0)

    # Measurement
    qc.measure(0, 0)

    return qc, theta, normalized

def simulate_your_code_classically(pixel_value, min_val, max_val, shots=1000):
    """Simulate your code classically"""
    from random import choices

    normalized = normalize_pixel_value(pixel_value, min_val, max_val)
    theta = isqr_encoding(normalized)

    a1 = np.cos(theta)
    a2 = np.sin(theta)
    i_state = np.array([[a1], [a2]])

    H = np.array([[1, 0], [0, 1]])

    cov = np.dot(H, i_state)
    si = cov * (1 / np.sqrt(1))

    alpha = si[0][0]
    beta = si[1][0]

    results = []
    for _ in range(shots):
        e = choices([alpha, beta], weights=[alpha**2, beta**2], k=1)
        if e[0] == beta:
            results.append(1)  # Maps to 255 in your code
        else:
            results.append(0)  # Maps to 0 in your code

    # Calculate probabilities
    prob_1 = sum(results) / shots
    prob_0 = 1 - prob_1

    return prob_0, prob_1, alpha**2, beta**2

def compare_quantum_classical(pixel_value, min_val=0, max_val=255, shots=1000):
    """Compare quantum circuit with your classical code"""

    print("=" * 70)
    print(f"COMPARISON FOR PIXEL VALUE: {pixel_value}")
    print("=" * 70)

    qc, theta, normalized = create_quantum_circuit(
        pixel_value, min_val, max_val
    )

    simulator = AerSimulator()
    tqc = transpile(qc, simulator)
    job = simulator.run(tqc, shots=shots)
    result = job.result()
    counts = result.get_counts()

    quantum_prob_0 = counts.get('0', 0) / shots
    quantum_prob_1 = counts.get('1', 0) / shots

    classical_prob_0, classical_prob_1, alpha_sq, beta_sq = simulate_your_code_classically(
        pixel_value, min_val, max_val, shots
    )

    # Display comparison
    print(f"\n{'Parameter':<20} {'Quantum':<15} {'Your Code':<15} {'Difference':<10}")
    print("-" * 60)
    print(f"{'P(|0⟩)':<20} {quantum_prob_0:<15.4f} {classical_prob_0:<15.4f} {abs(quantum_prob_0 - classical_prob_0):<10.4f}")
    print(f"{'P(|1⟩)':<20} {quantum_prob_1:<15.4f} {classical_prob_1:<15.4f} {abs(quantum_prob_1 - classical_prob_1):<10.4f}")
    print(f"{'Alpha^2 (cos²θ)':<20} {'-':<15} {alpha_sq:<15.4f} {'-':<10}")
    print(f"{'Beta^2 (sin²θ)':<20} {'-':<15} {beta_sq:<15.4f} {'-':<10}")

    # Show quantum circuit
    print(f"\nQuantum Circuit (replicating your code):")
    print(qc.draw(output='text'))

    # Generate QASM
    qasm_str = qasm2.dumps(qc)
    print(f"\nQASM Code for IBM Quantum Composer:")
    print("-" * 40)
    print(qasm_str)

    # Save QASM to file
    filename = f"pixel_{pixel_value}_from_your_code.qasm"
    with open(filename, 'w') as f:
        f.write(qasm_str)
    print(f"\nQASM file saved as: {filename}")

    # Visualize probabilities
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))

    # Quantum vs Classical probabilities
    labels = ['P(|0⟩)', 'P(|1⟩)']
    quantum_probs = [quantum_prob_0, quantum_prob_1]
    classical_probs = [classical_prob_0, classical_prob_1]

    x = np.arange(len(labels))
    width = 0.35

    axes[0].bar(x - width/2, quantum_probs, width, label='Quantum Circuit', color='blue', alpha=0.7)
    axes[0].bar(x + width/2, classical_probs, width, label='Your Python Code', color='red', alpha=0.7)
    axes[0].set_xlabel('State')
    axes[0].set_ylabel('Probability')
    axes[0].set_title(f'Pixel {pixel_value}: Quantum vs Classical')
    axes[0].set_xticks(x)
    axes[0].set_xticklabels(labels)
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)

    # Bloch sphere visualization (simplified)
    angles = np.linspace(0, 2 * np.pi, 100)
    axes[1].plot(np.cos(angles), np.sin(angles), 'k-', alpha=0.3)  # Circle

    # State vector
    state_x = np.cos(theta)  # |0⟩ amplitude (real part for visualization)
    state_y = np.sin(theta)  # |1⟩ amplitude (imaginary part for visualization)

    axes[1].arrow(0, 0, state_x, state_y, head_width=0.05, head_length=0.1, fc='red', ec='red')
    axes[1].scatter([state_x], [state_y], color='red', s=100, zorder=5)
    axes[1].text(state_x + 0.1, state_y + 0.1, f'|ψ⟩ = cos(θ)|0⟩ + sin(θ)|1⟩\nθ = {theta:.3f} rad', fontsize=10)

    axes[1].set_xlabel('|0⟩ component (cos θ)')
    axes[1].set_ylabel('|1⟩ component (sin θ)')
    axes[1].set_title('Quantum State on Bloch Sphere (projection)')
    axes[1].grid(True, alpha=0.3)
    axes[1].axis('equal')

    plt.tight_layout()
    plt.show()

    return qc, quantum_prob_0, quantum_prob_1, classical_prob_0, classical_prob_1

def generate_qasm_for_multiple_pixels(pixel_values, min_val=0, max_val=255):
    """Generate QASM files for multiple pixel values"""

    print("=" * 70)
    print("GENERATING QASM FILES FROM YOUR CODE")
    print("=" * 70)

    all_qasm = []

    for pixel in pixel_values:
        print(f"\nProcessing pixel value: {pixel}")

        # Create circuit from your code
        qc, theta, normalized = create_quantum_circuit(pixel, min_val, max_val)

        # Generate QASM
        qasm_str = qasm2.dumps(qc)

        # Save to file
        filename = f"pixel_{pixel}_your_code.qasm"
        with open(filename, 'w') as f:
            f.write(qasm_str)

        print(f"  Saved: {filename}")
        print(f"  Theta: {theta:.4f} rad")
        print(f"  Circuit: {qc.num_qubits} qubit, {qc.depth()} depth")

        all_qasm.append((pixel, qasm_str, filename))

    return all_qasm

def create_complete_qasm_example():
    """Create a complete QASM example with comments explaining your code"""

    qasm_template = """// Quantum Circuit Replicating ISQR Encoding 
OPENQASM 2.0;
include "qelib1.inc";

// Parameters (example for pixel value 128, range 0-255)
// theta = 2.0 * acos(sqrt((128 - 0) / (255 - 0))) ≈ 1.5708 rad

qreg q[1];
creg c[1];

// Initialize to |0>


// Apply Ry(2*theta) to create: cos(theta)|0> + sin(theta)|1>
// For pixel=128: theta≈1.5708, so 2*theta≈3.1416
ry(3.141592653589793) q[0];  // pi radians for pixel 128

// Measurement 
measure q[0] -> c[0];
"""

    print("=" * 70)
    print("COMPLETE QASM TEMPLATE WITH EXPLANATION")
    print("=" * 70)
    print(qasm_template)

    # Save to file
    with open("isqr_template_your_code.qasm", "w") as f:
        f.write(qasm_template)

    print(f"\nTemplate saved to: isqr_template_your_code.qasm")

    return qasm_template

def main():
    """Main function - ISQR code as quantum circuits"""

    print("=" * 70)
    print("QUANTUM CIRCUITS FROM ISQR CODE")
    print("=" * 70)

    # Test with different pixel values
    test_pixels = [0, 64, 128, 192, 255]

    results = []
    for pixel in test_pixels:
        print(f"\n{'='*70}")
        print(f"PIXEL VALUE: {pixel}")
        print(f"{'='*70}")

        qc, q0, q1, c0, c1 = compare_quantum_classical(pixel, shots=1000)
        results.append((pixel, q0, q1, c0, c1))

    # Generate QASM files for all test pixels
    print(f"\n{'='*70}")
    print("GENERATING ALL QASM FILES")
    print(f"{'='*70}")

    all_qasm = generate_qasm_for_multiple_pixels(test_pixels)

    # Create complete template
    create_complete_qasm_example()

    # Summary
    print(f"\n{'='*70}")
    print("SUMMARY")
    print(f"{'='*70}")

    print(f"\n{'Pixel':<10} {'Quantum P(|0⟩)':<15} {'Quantum P(|1⟩)':<15} {'Classical P(|1⟩)':<15}")
    print("-" * 60)
    for pixel, q0, q1, c0, c1 in results:
        print(f"{pixel:<10} {q0:<15.4f} {q1:<15.4f} {c1:<15.4f}")

    print(f"\nGenerated QASM Files:")
    for pixel, _, filename in all_qasm:
        print(f"  pixel_{pixel}_your_code.qasm")
    print(f"  isqr_template_your_code.qasm")

    print(f"\nHow to use in IBM Quantum Composer:")
    print(f"  1. Go to: https://quantum-computing.ibm.com/composer")
    print(f"  2. Create new circuit")
    print(f"  3. Click 'Import' and paste QASM code")
    print(f"  4. To change pixel value, modify the ry() gate angle")
    print(f"  5. Run on simulator or real quantum hardware")

    # Mathematical explanation
    print(f"\n{'='*70}")
    print("MATHEMATICAL EQUIVALENCE")
    print(f"{'='*70}")

    print(f"""
Quantum circuit equivalent:
1. Initialize |0⟩ state
2. Apply Ry(2θ) gate: Ry(2θ)|0⟩ = cos(θ)|0⟩ + sin(θ)|1⟩
3. Measure in computational basis

The Ry gate matrix:
Ry(θ) = [[cos(θ/2), -sin(θ/2)],
         [sin(θ/2),  cos(θ/2)]]

For Ry(2θ)|0⟩:
  = [[cos(θ), -sin(θ)], * |0⟩
     [sin(θ),  cos(θ)]]
  = cos(θ)|0⟩ + sin(θ)|1⟩

EXACTLY matches your state vector!
    """)

if __name__ == "__main__":
    main()